In [ ]:
import pandas as pd
import numpy as np
import scipy

from pandas_datareader import data, wb

import sys
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
from pandas.tseries.offsets import *

In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
#
# Real ticker data
#
start = datetime.datetime(2006, 1, 1)
end = datetime.datetime.now()

#price_data = data.DataReader('AAPL', 'yahoo', start, end)
#adjclose_series = price_data['Adj Close']
#adjclose_series.name = 'close'

#df = pd.DataFrame(adjclose_series)
#
#

strategyname = 'strategy_880131'
d, info = matlab.loaddata('../../mat/'+strategyname+'.mat')

df = pd.DataFrame()
df['close'] = d.exo

box_size = 50

box_start = 0
box_start_idx = None

box_end = 0
box_quantity = 0

temp_l = []

column_flag = None


reversal_multiplier = 3
reversal_value = box_size * reversal_multiplier

reversal_flag = False

column_index = 0

# Simple point and figure algorihtm

for i in range(len(df)):
    if box_start == 0:
        box_start = df.close[i]

    else:
        box_start = box_start
        price_move = df.close[i] - box_start

        # First of all we need to set box size.
        # Then calculate price movement.
        # If price movement is more or equal than box size - renko bar(or bars) will be added

        if np.abs(price_move) >= box_size:

            # After we calculate box_quantity(price move divided by box size)
            # This number defines how much pnf boxes will be registred
            box_quantity = np.int32(np.floor(np.abs(price_move / box_size)))
            box_date = df.close.index[i]

            if column_flag == 'x' and reversal_flag == False:

                # Reversal check
                if price_move < 0 and price_move <= -reversal_value:
                    reversal_flag = True
                    box_start = temp_l[-1]['open']
                    

                elif price_move > 0:
                    reversal_flag = False

                    for b in range(int(box_quantity)):
                            d = {'date': box_date, 'open': box_end, 'close': box_end + box_size,
                                 'type': column_flag, 'column_index': column_index}

                            box_end = d['close']
                            temp_l.append(d)

            if column_flag == 'o' and reversal_flag == False:

                # Reversal check
                if price_move > 0 and price_move >= reversal_value:
                    reversal_flag = True
                    box_start = temp_l[-1]['open']
                    

                elif price_move < 0:
                    reversal_flag = False

                    for b in range(int(box_quantity)):
                        d = {'date': box_date, 'open': box_end, 'close': box_end - box_size,
                             'type': column_flag, 'column_index': column_index}

                        box_end = d['close']
                        temp_l.append(d)

            if column_flag == None and reversal_flag == False:

                # Adding first column

                if price_move > 0:
                    column_flag = 'x'

                if price_move < 0:
                    column_flag = 'o'

                for b in range(int(box_quantity)):
                    if column_flag == 'x':

                        if box_end == 0:
                            d = {'date': box_date, 'open': box_start, 'close': box_start + box_size, 'type': column_flag,
                                 'column_index': column_index}

                            box_end = d['close']
                            temp_l.append(d)

                        else:
                            d = {'date': box_date, 'open': box_end, 'close': box_end + box_size, 'type': column_flag,
                                 'column_index': column_index}

                            box_end = d['close']
                            temp_l.append(d)

                    if column_flag == 'o':

                        if box_end == 0:
                            d = {'date': box_date, 'open': box_start, 'close': box_start - box_size,
                                 'type': column_flag, 'column_index': column_index}

                            box_end = d['close']
                            temp_l.append(d)

                        else:
                            d = {'date': box_date, 'open': box_end, 'close': box_end - box_size, 'type': column_flag,
                                 'column_index': column_index}

                            box_end = d['close']
                            temp_l.append(d)

            # Handling the reversals
            if reversal_flag == True:
                column_index += 1
                
                if column_flag == 'x':

                    column_flag = 'o'

                    for b in range(int(box_quantity)):
                            d = {'date': box_date, 'open': box_end, 'close': box_end - box_size,
                                 'type': column_flag, 'column_index': column_index}

                            box_end = d['close']
                            temp_l.append(d)

                elif column_flag == 'o':

                    column_flag = 'x'

                    for b in range(int(box_quantity)):
                        d = {'date': box_date, 'open': box_end, 'close': box_end + box_size,
                             'type': column_flag, 'column_index': column_index}

                        box_end = d['close']
                        temp_l.append(d)

                reversal_flag = False
        try:
            box_start = temp_l[-1]['close']
            
        except IndexError:
            box_start = df.close[i]
            
pnf_df = pd.DataFrame(temp_l)                
pnf_df.index = pnf_df.column_index

pnf_last_column_value_df = pnf_df.groupby(pnf_df.index).last()
pnf_first_column_value_df = pnf_df.groupby(pnf_df.index).first()

In [ ]:
pnf_df.close.plot()
pnf_df.o_col_downmove_count.dropna().plot(secondary_y=True)
pnf_df.x_col_downmove_count.dropna().plot(secondary_y=True)

# Patterns

In [ ]:
# New x column

pnf_df['new_x_col'] = (pnf_df.type == 'x') & (pnf_df.type.shift(1) == 'o')
pnf_df['new_o_col'] = (pnf_df.type == 'o') & (pnf_df.type.shift(1) == 'x')

In [ ]:
pnf_df['tripple_top'] = pnf_last_column_value_df[pnf_last_column_value_df.type == 'x'].close == \
                            pnf_last_column_value_df[pnf_last_column_value_df.type == 'x'].close.shift(1)

pnf_df['tripple_top'] = pnf_df['tripple_top'].fillna(False)

pnf_df['tripple_top_price_level'] = pnf_df[pnf_df.tripple_top == True].groupby(pnf_df[pnf_df.tripple_top == True].index).last().close
pnf_df['tripple_top_price_level'] = pnf_df['tripple_top_price_level'].ffill()

pnf_df['tripple_top_breakout'] =((pnf_df.close == pnf_df.tripple_top_price_level) & (pnf_df.close.shift(1) < pnf_df.tripple_top_price_level) 
                                & (pnf_df.tripple_top==False))

In [ ]:
pnf_df['tripple_bot'] = pnf_last_column_value_df[pnf_last_column_value_df.type == 'o'].close == \
                            pnf_last_column_value_df[pnf_last_column_value_df.type == 'o'].close.shift(1)
    
pnf_df['tripple_bot'] = pnf_df['tripple_bot'].fillna(False)

pnf_df['tripple_bot_price_level'] = pnf_df[pnf_df.tripple_bot == True].groupby(pnf_df[pnf_df.tripple_bot == True].index).last().close
pnf_df['tripple_bot_price_level'] = pnf_df['tripple_bot_price_level'].ffill()

pnf_df['tripple_bot_breakout'] =((pnf_df.close == pnf_df.tripple_bot_price_level) & (pnf_df.close.shift(1) > pnf_df.tripple_bot_price_level) 
                                & (pnf_df.tripple_bot==False))

## Removing a duplicated breakouts
### Optional, because these breakouts can be treated as support/resistance levels breakouts

In [ ]:
tripple_top_breakout_dup = pnf_df[pnf_df.tripple_top_breakout == True]['tripple_top_price_level'].duplicated() 

pnf_df.loc[tripple_top_breakout_dup[tripple_top_breakout_dup == True].index, 'tripple_top_breakout'] = pnf_df.tripple_top_breakout.ix[
    tripple_top_breakout_dup[tripple_top_breakout_dup == True].index].replace(True,False)

tripple_bot_breakout_dup = pnf_df[pnf_df.tripple_top_breakout == True]['tripple_top_price_level'].duplicated() 

pnf_df.loc[tripple_bot_breakout_dup[tripple_bot_breakout_dup == True].index, 'tripple_bot_breakout'] = pnf_df.tripple_bot_breakout.ix[
    tripple_bot_breakout_dup[tripple_bot_breakout_dup == True].index].replace(True,False)

## Consecutive column price up/down move

In [ ]:
up_move_count = [0]
up_move_counter = 0

down_move_count = [0]
down_move_counter = 0

same_move_count = [0]
same_move_counter = 0

x_col_upmove = pnf_last_column_value_df[pnf_last_column_value_df.type == 'x'].close > \
                            pnf_last_column_value_df[pnf_last_column_value_df.type == 'x'].close.shift(1)
    
x_col_downmove = pnf_last_column_value_df[pnf_last_column_value_df.type == 'x'].close < \
                            pnf_last_column_value_df[pnf_last_column_value_df.type == 'x'].close.shift(1)
    
x_col_samemove = pnf_last_column_value_df[pnf_last_column_value_df.type == 'x'].close < \
                            pnf_last_column_value_df[pnf_last_column_value_df.type == 'x'].close.shift(1)
    
for i in x_col_upmove.index.unique():
    if x_col_upmove[i] == True and x_col_upmove.shift(1)[i] == False:
        up_move_counter = 1
        up_move_count.append(up_move_counter)
        
    elif x_col_upmove[i] == True and x_col_upmove.shift(1)[i] == True:
        up_move_counter += 1
        up_move_count.append(up_move_counter)
        
    elif x_col_upmove[i] == False and x_col_upmove.shift(1)[i] == True:
        up_move_counter = 0
        up_move_count.append(up_move_counter)
        
    elif x_col_upmove[i] == False and x_col_upmove.shift(1)[i] == False:
        up_move_counter = 0
        up_move_count.append(up_move_counter)
      
    
    
    if x_col_downmove[i] == True and x_col_downmove.shift(1)[i] == False:
        down_move_counter = 1
        down_move_count.append(down_move_counter)
        
    elif x_col_downmove[i] == True and x_col_downmove.shift(1)[i] == True:
        down_move_counter += 1
        down_move_count.append(down_move_counter)
        
    elif x_col_downmove[i] == False and x_col_downmove.shift(1)[i] == True:
        down_move_counter = 0
        down_move_count.append(down_move_counter)
        
    elif x_col_downmove[i] == False and x_col_downmove.shift(1)[i] == False:
        down_move_counter = 0
        down_move_count.append(down_move_counter)  
      
    
    
    if x_col_samemove[i] == True and x_col_samemove.shift(1)[i] == False:
        same_move_counter = 1
        same_move_count.append(same_move_counter)
        
    elif x_col_samemove[i] == True and x_col_samemove.shift(1)[i] == True:
        same_move_counter += 1
        same_move_count.append(same_move_counter)
        
    elif x_col_samemove[i] == False and x_col_samemove.shift(1)[i] == True:
        same_move_counter = 0
        same_move_count.append(same_move_counter)
        
    elif x_col_samemove[i] == False and x_col_samemove.shift(1)[i] == False:
        same_move_counter = 0
        same_move_count.append(same_move_counter)
        
pnf_df['x_col_upmove_count'] = pd.Series(up_move_count, index=x_col_upmove.index)
pnf_df['x_col_downmove_count'] = pd.Series(down_move_count, index=x_col_upmove.index)
pnf_df['x_col_samemove_count'] = pd.Series(same_move_count, index=x_col_upmove.index)

In [ ]:
up_move_count = [0]
up_move_counter = 0

down_move_count = [0]
down_move_counter = 0

same_move_count = [0]
same_move_counter = 0

o_col_upmove = pnf_last_column_value_df[pnf_last_column_value_df.type == 'o'].close > \
                            pnf_last_column_value_df[pnf_last_column_value_df.type == 'o'].close.shift(1)
    
o_col_downmove = pnf_last_column_value_df[pnf_last_column_value_df.type == 'o'].close < \
                            pnf_last_column_value_df[pnf_last_column_value_df.type == 'o'].close.shift(1)
    
o_col_samemove = pnf_last_column_value_df[pnf_last_column_value_df.type == 'o'].close < \
                            pnf_last_column_value_df[pnf_last_column_value_df.type == 'o'].close.shift(1)
    
for i in o_col_upmove.index.unique():
    if o_col_upmove[i] == True and o_col_upmove.shift(1)[i] == False:
        up_move_counter = 1
        up_move_count.append(up_move_counter)
        
    elif o_col_upmove[i] == True and o_col_upmove.shift(1)[i] == True:
        up_move_counter += 1
        up_move_count.append(up_move_counter)
        
    elif o_col_upmove[i] == False and o_col_upmove.shift(1)[i] == True:
        up_move_counter = 0
        up_move_count.append(up_move_counter)
        
    elif o_col_upmove[i] == False and o_col_upmove.shift(1)[i] == False:
        up_move_counter = 0
        up_move_count.append(up_move_counter)
      
    
    
    if o_col_downmove[i] == True and o_col_downmove.shift(1)[i] == False:
        down_move_counter = 1
        down_move_count.append(down_move_counter)
        
    elif o_col_downmove[i] == True and o_col_downmove.shift(1)[i] == True:
        down_move_counter += 1
        down_move_count.append(down_move_counter)
        
    elif o_col_downmove[i] == False and o_col_downmove.shift(1)[i] == True:
        down_move_counter = 0
        down_move_count.append(down_move_counter)
        
    elif o_col_downmove[i] == False and o_col_downmove.shift(1)[i] == False:
        down_move_counter = 0
        down_move_count.append(down_move_counter)  
      
    
    
    if o_col_samemove[i] == True and o_col_samemove.shift(1)[i] == False:
        same_move_counter = 1
        same_move_count.append(same_move_counter)
        
    elif o_col_samemove[i] == True and o_col_samemove.shift(1)[i] == True:
        same_move_counter += 1
        same_move_count.append(same_move_counter)
        
    elif o_col_samemove[i] == False and o_col_samemove.shift(1)[i] == True:
        same_move_counter = 0
        same_move_count.append(same_move_counter)
        
    elif o_col_samemove[i] == False and o_col_samemove.shift(1)[i] == False:
        same_move_counter = 0
        same_move_count.append(same_move_counter)
        
pnf_df['o_col_upmove_count'] = pd.Series(up_move_count, index=o_col_upmove.index)
pnf_df['o_col_downmove_count'] = pd.Series(down_move_count, index=o_col_upmove.index)
pnf_df['o_col_samemove_count'] = pd.Series(same_move_count, index=o_col_upmove.index)

## Column box count

In [ ]:
for i in pnf_df.index.unique():
    pnf_df.loc[i,'box_count'] = pnf_df.close.ix[i].count()

In [ ]:
bull_fail = ((pnf_df.box_count == reversal_multiplier) & (pnf_df.type == 'x')).groupby(((pnf_df.box_count == reversal_multiplier)
                                                                                   & (pnf_df.type == 'x')).index).last()

pnf_first_column_value_df['bullish_failure'] = (bull_fail.shift(1) == True).groupby((bull_fail.shift(1) == True).index).first()


bear_fail = ((pnf_df.box_count == reversal_multiplier) & (pnf_df.type == 'o')).groupby(((pnf_df.box_count == reversal_multiplier)
                                                                                   & (pnf_df.type == 'o')).index).last()

pnf_first_column_value_df['bearish_failure'] = (bear_fail.shift(1) == True).groupby((bear_fail.shift(1) == True).index).first()

In [ ]:
pnf_df.close.plot()
pnf_df.box_count.plot(secondary_y=True)
pnf_df.box_count.rolling(len(pnf_last_column_value_df.close) * 0.25).mean().plot(secondary_y=True)

## Projection count

In [ ]:
pnf_df['pc_low'] = pnf_last_column_value_df.close == pnf_last_column_value_df.close.rolling(len(pnf_last_column_value_df.close) * 0.05).min()

pnf_df['pc_low_lastprice'] = pnf_last_column_value_df[pnf_last_column_value_df.close == 
                                                      pnf_last_column_value_df.close.rolling(len(pnf_last_column_value_df.close) * 0.05).min()].close

pnf_df['pc_low_lastprice'] = pnf_df['pc_low_lastprice'].ffill()

pnf_df['pc_low_boxcount'] = np.zeros_like(pnf_df.index)


for i in pnf_df[(pnf_df.pc_low == False) & (pnf_df.pc_low.shift(1) == True)].index.unique():
    pnf_df.loc[i,'pc_low_boxcount'] = pnf_df.ix[i].close.count()

pnf_df['pc_low_entry'] = (pnf_df.pc_low_boxcount.shift(1) != 0) & (pnf_df.pc_low_boxcount == 0) & (pnf_df.pc_low != True) & \
                            (pnf_df.index != 0)

for i in pnf_df[(pnf_df.pc_low_boxcount != 0)].index.unique():
    pnf_df.loc[i,'pc_low_profittarget'] = (box_size * reversal_multiplier * pnf_df.pc_low_boxcount.ix[i]) + pnf_df.pc_low_lastprice.ix[i] 
    
pnf_df['pc_low_profittarget'] = pnf_df['pc_low_profittarget'].ffill()

In [ ]:
pnf_df['pc_high'] = pnf_last_column_value_df.close == pnf_last_column_value_df.close.rolling(len(pnf_last_column_value_df.close) * 0.05).max()

pnf_df['pc_high_lastprice'] = pnf_last_column_value_df[pnf_last_column_value_df.close == 
                                                      pnf_last_column_value_df.close.rolling(len(pnf_last_column_value_df.close) * 0.05).max()].close

pnf_df['pc_high_lastprice'] = pnf_df['pc_high_lastprice'].ffill()

pnf_df['pc_high_boxcount'] = np.zeros_like(pnf_df.index)


for i in pnf_df[(pnf_df.pc_high == False) & (pnf_df.pc_high.shift(1) == True)].index.unique():
    pnf_df.loc[i,'pc_high_boxcount'] = pnf_df.ix[i].close.count()

pnf_df['pc_high_entry'] = (pnf_df.pc_high_boxcount.shift(1) != 0) & (pnf_df.pc_high_boxcount == 0) & (pnf_df.pc_high != True) & \
                            (pnf_df.index != 0)


for i in pnf_df[(pnf_df.pc_high_boxcount != 0)].index.unique():
    pnf_df.loc[i,'pc_high_profittarget'] =  pnf_df.pc_high_lastprice.ix[i] - (box_size * reversal_multiplier * pnf_df.pc_high_boxcount.ix[i]) 
    
pnf_df['pc_high_profittarget'] = pnf_df['pc_high_profittarget'].ffill()

## Local high/low

In [ ]:
pnf_first_column_value_df['local_high'] = (pnf_last_column_value_df.close == pnf_last_column_value_df.close.rolling(len(
                                            pnf_last_column_value_df.close) * 0.05).max()).shift(1) == True


pnf_first_column_value_df['local_low'] = (pnf_last_column_value_df.close == pnf_last_column_value_df.close.rolling(
                                            len(pnf_last_column_value_df.close) * 0.05).min()).shift(1) == True

In [ ]:
pnf_first_column_value_df

In [ ]:
figsize(20,10)

pnf_df.close.plot()
#pnf_df.local_high.plot(secondary_y=True)
#pnf_df.box_count.rolling(len(pnf_last_column_value_df.close) * 0.5).mean().plot(secondary_y=True)

pnf_last_column_value_df[(pnf_last_column_value_df.close == pnf_last_column_value_df.close.rolling(len(pnf_last_column_value_df.close) * 0.01).max()
                         ) == True].close.plot()

((pnf_last_column_value_df.close == pnf_last_column_value_df.close.rolling(len(pnf_last_column_value_df.close) * 0.01).max()) == True).plot(secondary_y=True)

In [ ]:
pnf_df[pnf_df.local_high == True]

In [ ]:
pnf_df.close.plot()
pnf_df.pc_high_entry.plot(secondary_y=True)
pnf_df.pc_high_profittarget.plot()

In [ ]:
pnf_first_column_value_df

In [ ]:
d, info = matlab.loaddata('../../mat/'+strategyname+'.mat')

ddd = d.join(pnf_df.set_index('date')[['new_x_col', 'new_o_col', 'tripple_bot_breakout', 'tripple_top_breakout']])
ddd = ddd.join(pnf_first_column_value_df.set_index('date')[['bearish_failure', 'bearish_failure', 'local_high', 'local_low']])

In [ ]:
ddd.local_low == True

In [ ]:
ddd.exo.ix['2016-04-15':'2016-04-30'].plot()

In [ ]:
var = pnf_df[(pnf_df.close == pnf_df.close.rolling(len(pnf_df.close) * 0.01).min()) == True]
#var.index.unique()

#var = pnf_df[(pnf_df.close == pnf_df.close.expanding().max()) == True]

for i in var.index.unique():
    
    if i > 0:
                  
        pnf_df['test_trendline_'+str(i)] = np.array([np.NaN] * len(pnf_df.index))
        pnf_df.loc[i-1, 'test_trendline_'+str(i)] = pnf_df.close[i-1]

        pnf_df.loc[i, 'test_trendline_'+str(i)] = pnf_first_column_value_df.close[i]

        pnf_df['test_trendline_'+str(i)] = pnf_df['test_trendline_'+str(i)].groupby(
            pnf_df['test_trendline_'+str(i)].index).last().interpolate('barycentric')

In [ ]:
figsize(20,10)

pnf_first_column_value_df[pnf_first_column_value_df.type == 'o'].close.expanding().max().plot(legend=False)

pnf_first_column_value_df[pnf_first_column_value_df.type == 'o'].close.plot(legend=False)

In [ ]:
pnf_df[(pnf_df.date > '2015-12-31') & (pnf_df.date < '2016-12-31')]

In [ ]:
close_prices_occurance = pnf_last_column_value_df.close.groupby(pnf_last_column_value_df.close).count()
close_prices_occurance[close_prices_occurance == 4]

In [ ]:
pnf_df.close.plot()

axhline(-1786.8125)
#axhline(-146.5625)
#axhline(207.434017)
#axhline(-4920.0)

In [ ]:
pnf_df.close.plot()
#(pnf_df.close.rolling(len(pnf_df.close) * 0.05).min()).plot(secondary_y=True)
#pnf_df[(pnf_df.date > '2014-12-31') & (pnf_df.date < '2015-12-31')].close.expanding(min_periods=10).max().plot()
#(pnf_df.close == pnf_df[(pnf_df.date > '2014-12-31') & (pnf_df.date < '2015-12-31')].close.max()).plot(secondary_y=True)
#(pnf_first_column_value_df.close.expanding(min_periods=10).min()).plot()
pnf_df['px_low'].plot(secondary_y=True)

In [ ]:
pnf_df[pnf_df.close == pnf_df.close.max()]

In [ ]:
i=18

pnf_df['test_trendline'+str(i)] = np.array([np.NaN] * len(pnf_df.index))
pnf_df.loc[i-1, 'test_trendline'+str(i)] = pnf_df.close[i-1]
pnf_df.loc[i, 'test_trendline'+str(i)] = pnf_first_column_value_df.close[i]

pnf_df['test_trendline'+str(i)] = pnf_df['test_trendline'+str(i)].groupby(
    pnf_df['test_trendline'+str(i)].index).last().interpolate('barycentric')

#pnf_df.drop(['open', 'column_index'],1).plot(ylim=[30,220])
#(pnf_df.close == pnf_df.close.rolling(len(pnf_df.close) * 0.05).max()).plot(secondary_y=True)7
(pnf_df['test_trendline'+str(i)]).plot()
(pnf_df.close).plot()

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

# Create random data with numpy
import numpy as np

py.sign_in('tmqr', '8rctfyu197')


trace1 = go.Scatter(
    x = pnf_df[pnf_df.type == 'o'].index,
    y = pnf_df[pnf_df.type == 'o'].close,
    name='Down',
    mode = 'markers',
    marker = dict(symbol="o", size=5, color = 'rgba(255, 0, 0, 0.9)'),
    text = pnf_df[pnf_df.type == 'o'].date
)

trace2 = go.Scatter(
    x = pnf_df[pnf_df.type == 'x'].index,
    y = pnf_df[pnf_df.type == 'x'].close,
    name = 'Up',
    mode = 'markers',
    marker = dict(symbol="x", size=5, color = 'rgba(0, 255, 0, 0.9)'),
    text = pnf_df[pnf_df.type == 'x'].date
    ) 

xo = [trace1, trace2]

layout = go.Layout(
    title="Point And Figure",
    xaxis=dict( autotick=True, autorange=True
        )
    )
fig = go.Figure(data=xo, layout=layout)
# Plot and embed in ipython notebook!
py.iplot(fig, filename='basic-scatter')

In [ ]:
trace = go.Scatter(
    x = df.index,
    y = df.close
)

data = [trace]

# Plot and embed in ipython notebook!
py.iplot(data, filename='basic-line')

In [ ]:
pnf_df.ix[27]

In [ ]:
9

In [ ]:
34+23